# Swift vs Boto3 for Parallelisation with Dask

In [1]:
import boto3
import swiftclient
import os

In [2]:
s3_keys_path = '~/.keys/lsst_keys.json'
swift_keys_path = '~/.keys/lsst-swift-credentials.json'
creds = {}

#for boto3
with open(os.path.expanduser(s3_keys_path), 'r') as s3keys:
    for line in s3keys.readlines():
        if 'access_key' in line:
            creds['S3_ACCESS_KEY'] = line.split('"')[3]
        elif 'secret_key' in line:
            creds['S3_SECRET_KEY'] = line.split('"')[3]
creds['S3_HOST_URL'] = 'https://echo.stfc.ac.uk'

#for swift
with open(os.path.expanduser(swift_keys_path), 'r') as swiftkeys:
    for line in swiftkeys.readlines():
        if 'user' in line:
            creds['ST_USER'] = line.split('"')[3]
        elif 'secret_key' in line:
            creds['ST_KEY'] = line.split('"')[3]
creds['ST_AUTH'] = 'https://s3.echo.stfc.ac.uk/auth/1.0'

#Some Swift operations expect environment variables
#Here we just mirror the creds dict to the os.environ dict
for k, v in creds.items():
    os.environ[k] = v

## boto3 resource

In [3]:
session = boto3.Session(
    aws_access_key_id = creds['S3_ACCESS_KEY'],
    aws_secret_access_key = creds['S3_SECRET_KEY']
)

In [4]:
resource = session.resource(
    service_name = 's3',
    endpoint_url = creds['S3_HOST_URL']
)
resource

s3.ServiceResource()

In [5]:
# %%timeit
boto3_bucket_list = [ b.name for b in resource.buckets.all() ]

In [6]:
boto3_bucket_list

['DRP',
 'LSST-IR-FUSION',
 'LSST-IR-FUSION-Butlers',
 'LSST-IR-FUSION-TEST',
 'LSST-IR-FUSION-rdsip005',
 'LSST-IR-FUSION-testfromopenstack',
 'LSST-IR-FUSION_gen3_conversion',
 'dmu4',
 'lsst-dac',
 'lsst-drp-config',
 'lsst-test']

## Swift Connection

In [7]:
connection = swiftclient.Connection(
    user = creds['ST_USER'],
    key = creds['ST_KEY'],
    authurl = creds['ST_AUTH']
)

In [8]:
# %%timeit
containers = [ container['name'] for container in connection.get_account()[1] ]

In [9]:
containers

['DRP',
 'LSST-IR-FUSION',
 'LSST-IR-FUSION-Butlers',
 'LSST-IR-FUSION-TEST',
 'LSST-IR-FUSION-rdsip005',
 'LSST-IR-FUSION-testfromopenstack',
 'LSST-IR-FUSION_gen3_conversion',
 'dmu4',
 'lsst-dac',
 'lsst-drp-config',
 'lsst-test']

In [10]:
bucket = resource.Bucket('LSST-IR-FUSION-testfromopenstack')

In [11]:
# %%timeit
boto3_objects = [ obj.key for obj in bucket.objects.all() ]

In [12]:
boto3_objects

['dummy-lsst-backup.csv',
 'dummy/1/1_1.f',
 'dummy/1/1_2.f',
 'dummy/14/14_1.f',
 'dummy/14/14_2.f',
 'dummy/17/17_2.f',
 'dummy/18/18_1.f',
 'dummy/18/18_2.f',
 'dummy/2/2_1.f',
 'dummy/2/2_2.f',
 'dummy/21/21_1.f',
 'dummy/21/21_2.f',
 'dummy/30/30_1.f',
 'dummy/30/30_2.f',
 'dummy/31/31_1.f',
 'dummy/31/31_2.f',
 'dummy/36/36_1.f',
 'dummy/36/36_2.f',
 'dummy/44/44_1.f',
 'dummy/44/44_2.f',
 'dummy/46/46_1.f',
 'dummy/5/5_1.f',
 'dummy/5/5_2.f',
 'dummy/50/50_1.f',
 'dummy/50/50_2.f',
 'dummy/52/52_1.f',
 'dummy/52/52_2.f',
 'dummy/53/53_1.f',
 'dummy/53/53_2.f',
 'dummy/60/60_2.f',
 'dummy/67/67_1.f',
 'dummy/67/67_2.f',
 'dummy/7/7_1.f',
 'dummy/7/7_2.f',
 'dummy/72/72_1.f',
 'dummy/72/72_2.f',
 'dummy/8/8_1.f',
 'dummy/8/8_2.f',
 'dummy/81/81_1.f',
 'dummy/81/81_2.f',
 'dummy/87/87_1.f',
 'dummy/87/87_2.f',
 'dummy/88/88_1.f',
 'dummy/88/88_2.f',
 'dummy/97/97_1.f',
 'dummy/97/97_2.f',
 'dummy/98/98_1.f',
 'dummy/98/98_2.f',
 'dummy/collated_0.zip',
 'dummy/collated_0.zip.metada

In [13]:
# %%timeit
swift_objects = [ obj['name'] for obj in connection.get_container('LSST-IR-FUSION-testfromopenstack')[1] ]

In [14]:
swift_objects

['dummy-lsst-backup.csv',
 'dummy/1/1_1.f',
 'dummy/1/1_2.f',
 'dummy/14/14_1.f',
 'dummy/14/14_2.f',
 'dummy/17/17_2.f',
 'dummy/18/18_1.f',
 'dummy/18/18_2.f',
 'dummy/2/2_1.f',
 'dummy/2/2_2.f',
 'dummy/21/21_1.f',
 'dummy/21/21_2.f',
 'dummy/30/30_1.f',
 'dummy/30/30_2.f',
 'dummy/31/31_1.f',
 'dummy/31/31_2.f',
 'dummy/36/36_1.f',
 'dummy/36/36_2.f',
 'dummy/44/44_1.f',
 'dummy/44/44_2.f',
 'dummy/46/46_1.f',
 'dummy/5/5_1.f',
 'dummy/5/5_2.f',
 'dummy/50/50_1.f',
 'dummy/50/50_2.f',
 'dummy/52/52_1.f',
 'dummy/52/52_2.f',
 'dummy/53/53_1.f',
 'dummy/53/53_2.f',
 'dummy/60/60_2.f',
 'dummy/67/67_1.f',
 'dummy/67/67_2.f',
 'dummy/7/7_1.f',
 'dummy/7/7_2.f',
 'dummy/72/72_1.f',
 'dummy/72/72_2.f',
 'dummy/8/8_1.f',
 'dummy/8/8_2.f',
 'dummy/81/81_1.f',
 'dummy/81/81_2.f',
 'dummy/87/87_1.f',
 'dummy/87/87_2.f',
 'dummy/88/88_1.f',
 'dummy/88/88_2.f',
 'dummy/97/97_1.f',
 'dummy/97/97_2.f',
 'dummy/98/98_1.f',
 'dummy/98/98_2.f',
 'dummy/collated_0.zip',
 'dummy/collated_0.zip.metada

With a simple test of getting lists buckets and objects within a bucket, the boto3 API seems faster at this stage.

Now we'll try something abit more in-depth.

In [15]:
def get_metadata_boto3(key, bucket):
    if key.endswith('.zip'):
        try:
            metadata = str(bucket.Object(''.join([key,'.metadata'])).get()['Body'].read().decode('UTF-8'))
        except:
            return ''
        return metadata
    else:
        return ''

In [16]:
def get_metadata_swift(key, connection, container_name):
    metadata = None
    if key.endswith('.zip'):
        try:
            metadata = str(connection.get_object(container_name,''.join([key,'.metadata']))[1].decode('UTF-8'))
        except:
            return ''
        return metadata
    else:
        return ''

In [17]:
# %%timeit
metadata_boto3 = get_metadata_boto3('dummy/collated_7.zip', bucket)

In [18]:
metadata_boto3

'18/18_1.f|44/44_1.f|44/44_2.f'

In [19]:
# %%timeit
metadata_swift = get_metadata_swift('dummy/collated_7.zip', connection, 'LSST-IR-FUSION-testfromopenstack')

In [20]:
metadata_swift

'18/18_1.f|44/44_1.f|44/44_2.f'

Now switch to Pandas, so we can use DataFrame.apply() to run these functions on every row of a DataFrame

In [21]:
import pandas as pd

In [22]:
expanded_objects = []
for _ in range(10):
    expanded_objects.extend(boto3_objects)

In [23]:
len(expanded_objects)

800

In [24]:
objects_boto3 = pd.DataFrame.from_dict({'key':expanded_objects},dtype=str)

In [25]:
objects_boto3

,key
0,dummy-lsst-backup.csv
1,dummy/1/1_1.f
2,dummy/1/1_2.f
3,dummy/14/14_1.f
4,dummy/14/14_2.f
...,...
795,dummy/collated_65.zip.metadata
796,dummy/collated_66.zip
797,dummy/collated_66.zip.metadata
798,dummy/collated_7.zip


In [26]:
# %%timeit
objects_boto3['metadata'] = objects_boto3['key'].apply(get_metadata_boto3, bucket=bucket)

In [27]:
objects_boto3[objects_boto3['key'].str.endswith('.zip')]

,key,metadata
48,dummy/collated_0.zip,7/7_1.f|7/7_2.f|98/98_2.f
50,dummy/collated_1.zip,98/98_1.f|53/53_2.f|53/53_1.f
52,dummy/collated_2.zip,50/50_2.f|50/50_1.f|67/67_2.f
54,dummy/collated_3.zip,67/67_1.f|2/2_2.f|2/2_1.f
56,dummy/collated_4.zip,8/8_1.f|8/8_2.f|87/87_2.f
...,...,...
790,dummy/collated_63.zip,81/81_2.f|52/52_1.f|52/52_2.f
792,dummy/collated_64.zip,31/31_1.f|31/31_2.f|30/30_2.f
794,dummy/collated_65.zip,30/30_1.f|36/36_2.f|36/36_1.f
796,dummy/collated_66.zip,14/14_2.f|14/14_1.f


In [28]:
objects_swift = pd.DataFrame.from_dict({'key':expanded_objects},dtype=str)

In [29]:
objects_swift

,key
0,dummy-lsst-backup.csv
1,dummy/1/1_1.f
2,dummy/1/1_2.f
3,dummy/14/14_1.f
4,dummy/14/14_2.f
...,...
795,dummy/collated_65.zip.metadata
796,dummy/collated_66.zip
797,dummy/collated_66.zip.metadata
798,dummy/collated_7.zip


In [30]:
# %%timeit
objects_swift['metadata'] = objects_swift['key'].apply(get_metadata_swift, connection=connection, container_name='LSST-IR-FUSION-testfromopenstack')

In [31]:
objects_swift[objects_swift['key'].str.endswith('.zip')]

,key,metadata
48,dummy/collated_0.zip,7/7_1.f|7/7_2.f|98/98_2.f
50,dummy/collated_1.zip,98/98_1.f|53/53_2.f|53/53_1.f
52,dummy/collated_2.zip,50/50_2.f|50/50_1.f|67/67_2.f
54,dummy/collated_3.zip,67/67_1.f|2/2_2.f|2/2_1.f
56,dummy/collated_4.zip,8/8_1.f|8/8_2.f|87/87_2.f
...,...,...
790,dummy/collated_63.zip,81/81_2.f|52/52_1.f|52/52_2.f
792,dummy/collated_64.zip,31/31_1.f|31/31_2.f|30/30_2.f
794,dummy/collated_65.zip,30/30_1.f|36/36_2.f|36/36_1.f
796,dummy/collated_66.zip,14/14_2.f|14/14_1.f


Swift and boto3 still take around the same amount of time to give us the metadata for all of the zip objects using Pandas and our metadata functions to fetch it.
Both have established connection (or resource) objects that are already authenticated with the server.
Both are bound by the internet connection speed.

Let's try to parallelise them using Dask

## Dask parallelisation

Note: this will use a parallel version of the pandas functionality, _not_ parallel versions of the boto3 or Swift functions.

In [32]:
from dask import dataframe as ddf
import distributed

In [33]:
client = distributed.Client()

In [34]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 7.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44861,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 7.43 GiB
Comm: tcp://127.0.0.1:40709,Total threads: 3
Dashboard: http://127.0.0.1:34985/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:32887,


In [35]:
objects_swift = pd.DataFrame.from_dict({'key':expanded_objects},dtype=str)

In [36]:
objects_swift_dask = ddf.from_pandas(objects_swift, npartitions=len(expanded_objects)//12)

In [37]:
objects_swift_dask

,key
npartitions=66,
0,string
13,...
...,...
788,...
799,...


In [38]:
objects_swift_dask['metadata'] = objects_swift_dask['key'].apply(get_metadata_swift, connection=connection, container_name='LSST-IR-FUSION-testfromopenstack', meta=pd.Series(dtype=str))

In [39]:
objects_swift_dask

,key,metadata
npartitions=66,,
0,string,object
13,...,...
...,...,...
788,...,...
799,...,...


In [40]:
# %%timeit
objects_swift = objects_swift_dask.compute()

In [41]:
objects_swift

,key,metadata
0,dummy-lsst-backup.csv,
1,dummy/1/1_1.f,
2,dummy/1/1_2.f,
3,dummy/14/14_1.f,
4,dummy/14/14_2.f,
...,...,...
795,dummy/collated_65.zip.metadata,
796,dummy/collated_66.zip,14/14_2.f|14/14_1.f
797,dummy/collated_66.zip.metadata,
798,dummy/collated_7.zip,18/18_1.f|44/44_1.f|44/44_2.f


In [42]:
objects_boto3 = pd.DataFrame.from_dict({'key':expanded_objects},dtype=str)

In [43]:
objects_boto3_dask = ddf.from_pandas(objects_boto3, npartitions=len(expanded_objects)//12)

In [44]:
objects_boto3_dask

,key
npartitions=66,
0,string
13,...
...,...
788,...
799,...


In [45]:
# objects_boto3_dask['metadata'] = objects_boto3_dask['key'].apply(get_metadata_boto3, bucket=bucket, meta=pd.Series(dtype=str))

## Conclusion

- OpenStack Swift API is not 10 times faster than S3 API, it's about as fast.
- OpenStack python-swiftclient is not 10 times faster than boto3, it's about as fast.
- Using swiftclient with Pandas is not 10 times faster than using boto3 with Pandas, it's about as fast.
- Using swiftclient with Dask _is_ __at least__ 10 times faster than using either swiftclient or boto3 with Pandas.
- Using boto3 with Dask is impossible.

Therefore, the combination of swiftclient and Dask is __at least__ 10 times faster than the de facto industry standards of boto3 and Pandas, because Dask is faster than Pandas, and swiftclient objects __are__ _picklable_ and boto3 objects __are not__ _picklable_.


## Appendix

Minimal example of pickling.

In [46]:
import s3fs

In [47]:
for i in range(len(swift_objects)):
    print(i,swift_objects[i])

0 dummy-lsst-backup.csv
1 dummy/1/1_1.f
2 dummy/1/1_2.f
3 dummy/14/14_1.f
4 dummy/14/14_2.f
5 dummy/17/17_2.f
6 dummy/18/18_1.f
7 dummy/18/18_2.f
8 dummy/2/2_1.f
9 dummy/2/2_2.f
10 dummy/21/21_1.f
11 dummy/21/21_2.f
12 dummy/30/30_1.f
13 dummy/30/30_2.f
14 dummy/31/31_1.f
15 dummy/31/31_2.f
16 dummy/36/36_1.f
17 dummy/36/36_2.f
18 dummy/44/44_1.f
19 dummy/44/44_2.f
20 dummy/46/46_1.f
21 dummy/5/5_1.f
22 dummy/5/5_2.f
23 dummy/50/50_1.f
24 dummy/50/50_2.f
25 dummy/52/52_1.f
26 dummy/52/52_2.f
27 dummy/53/53_1.f
28 dummy/53/53_2.f
29 dummy/60/60_2.f
30 dummy/67/67_1.f
31 dummy/67/67_2.f
32 dummy/7/7_1.f
33 dummy/7/7_2.f
34 dummy/72/72_1.f
35 dummy/72/72_2.f
36 dummy/8/8_1.f
37 dummy/8/8_2.f
38 dummy/81/81_1.f
39 dummy/81/81_2.f
40 dummy/87/87_1.f
41 dummy/87/87_2.f
42 dummy/88/88_1.f
43 dummy/88/88_2.f
44 dummy/97/97_1.f
45 dummy/97/97_2.f
46 dummy/98/98_1.f
47 dummy/98/98_2.f
48 dummy/collated_0.zip
49 dummy/collated_0.zip.metadata
50 dummy/collated_1.zip
51 dummy/collated_1.zip.metadat

In [48]:
def get_metadata_s3fs(key, bucket_name, client):
    if key.endswith('.zip.metadata'):
        try:
            metadata = s3.cat(f'{bucket_name}/{key}').decode('utf-8')
        except:
            return ''
        return metadata
    else:
        return ''

s3 = s3fs.S3FileSystem(key=creds['S3_ACCESS_KEY'], secret=creds['S3_SECRET_KEY'], endpoint_url=creds['S3_HOST_URL'])


In [49]:
objects_s3fs_d = pd.DataFrame.from_dict({'key':expanded_objects},dtype=str)
objects_s3fs_d

,key
0,dummy-lsst-backup.csv
1,dummy/1/1_1.f
2,dummy/1/1_2.f
3,dummy/14/14_1.f
4,dummy/14/14_2.f
...,...
795,dummy/collated_65.zip.metadata
796,dummy/collated_66.zip
797,dummy/collated_66.zip.metadata
798,dummy/collated_7.zip


In [50]:
objects_s3fs_dask = ddf.from_pandas(objects_s3fs_d, npartitions=len(expanded_objects)//12)
objects_s3fs_dask

,key
npartitions=66,
0,string
13,...
...,...
788,...
799,...


In [51]:
objects_s3fs_dask['metadata'] = objects_s3fs_dask['key'].apply(get_metadata_s3fs, bucket_name='LSST-IR-FUSION-testfromopenstack', client=s3, meta=pd.Series(dtype=str))

In [52]:
get_metadata_s3fs('dummy/collated_66.zip.metadata', bucket_name='LSST-IR-FUSION-testfromopenstack', client=s3)

'14/14_2.f|14/14_1.f'

In [55]:
%%timeit
objects_s3fs_d = objects_s3fs_dask.compute()

2.03 s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
objects_s3fs_d['metadata']

0                                   
1                                   
2                                   
3                                   
4                                   
                   ...              
795    30/30_1.f|36/36_2.f|36/36_1.f
796                                 
797              14/14_2.f|14/14_1.f
798                                 
799    18/18_1.f|44/44_1.f|44/44_2.f
Name: metadata, Length: 800, dtype: object